# Retail SVD Recommender

In the previous section we multiplied the $\mathbf{u}_i$ and $\mathbf{v}_i$ column vectors to get a reduced rank approximation for all customers. In this section we will calculate the incremental recommendation $\mathbf{\tilde{r}}_i$ for a customer.

### Steps

1. Get the actual ratings from the rating table.
1. Use dot-products to compute the approximated ratings.
1. Take the difference of the actual and approximation to get the recomendation.

### Prerequsites

You should have run the [svd_2_calc.ipynb](svd_2_calc.ipynb) notebook to generate the vector tables.

In [48]:
# Local libraries should automatically reload
%reload_ext autoreload
%autoreload 1
%matplotlib inline

import sys 
sys.path.append('../KJIO')
import numpy as np
import pandas as pd
%aimport kodbc_io

pd.options.display.max_colwidth = 100

### Get actual purchases

Here we get the actual ratings of a customer. You can do this for any customer ID.

In [57]:
CUSTOMER_ID = 22632

_actual_df = kodbc_io.get_df("""
select
    am.product_parent,
    ap.product_title,
    am.star_rating
from amazon_matrix am
join amazon_products ap
    on am.product_parent = ap.product_parent
where am.customer_id = {}
order by am.product_parent
""".format(CUSTOMER_ID))

_actual_df = _actual_df.set_index('product_parent')
_actual_df

Connected to GPUdb ODBC Server (6.2.0.17.20180825221415)
Rows returned: 2


,product_title,star_rating
product_parent,,
274951190,UltraFire E17 1000 Lumen Flashlight Led Cree T6 XM-L Camping Torch Flash Light The Lamp With Mount,5
327072312,"SOG Tactical Tomahawk F01TN-CP - Hardcased Black Axe Head, GRN Handle, Nylon Sheath, 2.75"" Blade",5


### Get approximated ratings

Earlier we computed $\sqrt{\Sigma}\tilde{\mathbf{U}}$ and $\sqrt{\Sigma}\tilde{\mathbf{V}}^T$ which were saved to tables. Here we use SQL to compute the rating approximation for a single customer:

$\mathbf{\tilde{a}}_c^T = A_c = U_c \boldsymbol{\Sigma} \mathbf{V}^T $

We can re-factor the SVD equation considering just the row vectors of *U* and can calculate the approximation for a a row vector of *A* given a row vector as a combination of dot-products. Each scalar of $\mathbf{\tilde{a}}_c$ is obtained with a dot product of the customer vector $\mathbf{\tilde{u}}_c$ with is respective product vector $\mathbf{\tilde{v}}_p$.

$\mathbf{\tilde{a}}_c^T = (\sqrt{\sigma} \mathbf{\tilde{u}}_c^T) \cdot (\sqrt{\sigma} \mathbf{\tilde{v}}_p) $

In [58]:
_approx_df = kodbc_io.get_df("""
select top 10 
    iv.product_parent,
    ap.product_title,
    (cv.U0 * iv.V0)
    + (cv.U1 * iv.V1) 
    + (cv.U2 * iv.V2) 
    + (cv.U3 * iv.V3) 
    + (cv.U4 * iv.V4) 
    + (cv.U5 * iv.V5) 
    + (cv.U6 * iv.V6) 
    + (cv.U7 * iv.V7) 
    + (cv.U8 * iv.V8) 
    + (cv.U9 * iv.V9) 
    as item_rating
from svd_cust_vec as cv, svd_item_vec as iv
join amazon_products ap
    on ap.product_parent = iv.product_parent
where cv.customer_id = {}
order by item_rating desc
""".format(CUSTOMER_ID))

_approx_df = _approx_df.set_index('product_parent')
_approx_df.sort_values('item_rating', ascending=False)

Connected to GPUdb ODBC Server (6.2.0.17.20180825221415)
Rows returned: 10


,product_title,item_rating
product_parent,,
826484350,"Smith's PP1 Pocket Pal Multifunction Sharpener, Grey",0.140416
844836744,UltraFire? 1000 LM WF-502B CREE XM-L T6 5-Mode LED Flashlight Torch (With Batteries and Charger),0.068183
101240200,Streamlight Nano Light Miniature Keychain LED Flashlight,0.056900
669820393,Howard Leight Impact Sport OD Electric,0.056288
497942344,SE MH1047L Illuminated Multi-Power LED Head Magnifier,0.043255
601818824,SE FS374 All-Weather Emergency 2-IN-1 Fire Starter & Magnesium Fuel Bar (Everything you need to ...,0.042074
532704912,Streamlight 88030 ProTac 1L 275 Lumen Professional Tactical Flashlight with High/Low/Strobe w/ 1...,0.039515
672080706,SE MZ101B Helping Hand with Magnifying Glass,0.035845
359191662,SE 7503SD 3-Piece Power Extension Bit Set for Drills,0.034435


### Get recommended purchases

Here we generate recommendations which is the difference between the actual and apprximation which is the same as the recommendation error.

$
\mathbf{\tilde{r}}_c = \mathbf{\tilde{a}}_c - \mathbf{a}_c = \mathbf{\epsilon}_c
$

Consider that the approximation will contain the customers own ratings combined with other ratings from similar customers. When we subtract the customers actual ratings what remains are preferences from other similar customers. These results are sorted by highest rating so the most significant recommendations are listed at the top.

In [55]:
_recommended_df = _approx_df.loc[set(_approx_df.index) - set(_actual_df.index)]
_recommended_df.sort_values('item_rating', ascending=False).head(4)

,product_title,item_rating
product_parent,,
672479254,CH Hanson 0 Stud 4 Sure Magnetic Stud Finder,0.020658
497942344,SE MH1047L Illuminated Multi-Power LED Head Magnifier,0.015847
359191662,SE 7503SD 3-Piece Power Extension Bit Set for Drills,0.014154
655034668,CLC Custom Leathercraft 125 Handyman Flex Grip Work Gloves,0.013996
